In [28]:
import os
from time import time, localtime
import logging
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import numpy as np
# from utils import plot_graph
# import tensorflow_federated as tff
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras import losses, metrics, optimizers
import random
import pandas as pd
import sys
import matplotlib.pyplot as plt
import nest_asyncio
from pathlib import Path
# from tensorflow_federated.python.research.utils import checkpoint_manager
# import tensorflow_addons as tfa
nest_asyncio.apply()
SEED = 1337
tf.random.set_seed(SEED)

In [29]:
def most_frequent(List):
    return max(set(List), key=List.count)

In [30]:
test_dir = "/home/jovyan/dataset/GQUIC_small/Test/GQUIC_test_32.feather"
test = pd.read_feather(test_dir)
# print(test)
test.to_csv("/home/jovyan/dataset/GQUIC_small/Test/GQUIC_test_32.csv")
# print(test)


In [31]:
result = test.groupby('flow_id')['Label'].apply(list).to_dict()
# print(result, type(result))
flow_label = []
for flow in result:
    flow_label.append(most_frequent(result[flow]))

flow_label = np.array(flow_label)

In [32]:
true_test = test.drop('flow_id', axis=1)
# print(true_test)

In [33]:
x_test = true_test.drop('Label', axis=1).to_numpy()/255
# print(x_test)

In [34]:
x_test[20]

array([0.0627451 , 0.04705882, 0.49411765, 0.31372549, 0.51372549,
       0.43137255, 0.05098039, 0.80392157, 0.57647059, 0.69411765,
       0.3372549 , 0.96470588, 0.08627451, 0.79215686, 0.44705882,
       0.7254902 , 0.78823529, 0.97254902, 0.00784314, 0.09803922,
       0.6       , 0.64313725, 0.11372549, 0.18039216, 0.63137255,
       0.22352941, 0.70980392, 0.11764706, 0.75686275, 0.16470588,
       0.79607843, 0.1372549 ])

In [35]:
print(x_test.shape)
x_test = x_test.reshape(-1,20,128,1)
print(x_test.shape)

(55600, 32)
(695, 20, 128, 1)


In [36]:
x_test[20]

array([[[0.68627451],
        [0.59607843],
        [0.17254902],
        ...,
        [0.88627451],
        [0.88627451],
        [0.10196078]],

       [[0.28627451],
        [0.17647059],
        [0.45490196],
        ...,
        [0.32156863],
        [0.16470588],
        [0.27058824]],

       [[0.29803922],
        [0.10980392],
        [0.65490196],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       ...,

       [[0.63921569],
        [0.30588235],
        [0.03921569],
        ...,
        [0.58431373],
        [0.6627451 ],
        [0.        ]],

       [[0.80392157],
        [0.60392157],
        [0.89803922],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.91372549],
        [0.62352941],
        [0.48627451],
        ...,
        [0.63921569],
        [0.69411765],
        [0.31764706]]])

### Model Definition

In [37]:
model = tf.keras.models.load_model("/home/jovyan/dataset/model_2")

In [38]:
model

In [39]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 20, 128, 32)       832       
                                                                 
 conv2d_13 (Conv2D)          (None, 20, 128, 32)       25632     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 10, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_14 (Conv2D)          (None, 10, 64, 64)        18496     
                                                                 
 conv2d_15 (Conv2D)          (None, 10, 64, 64)        36928     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 5, 32, 64)        0         
 2D)                                                  

### Predict

#### Predict all

In [40]:
predictions = model.predict(x_test,verbose=2,use_multiprocessing=True,batch_size=64)
print(predictions)

11/11 - 1s - 637ms/epoch - 58ms/step
[[1.0626294e-08 9.9896431e-01 1.4558329e-06 1.0342860e-03]
 [7.0241816e-07 9.5604706e-01 1.2743822e-04 4.3824881e-02]
 [2.0174480e-06 6.6617328e-01 8.0142915e-04 3.3302328e-01]
 ...
 [1.7344541e-04 9.9575979e-01 3.1113498e-06 4.0637380e-03]
 [2.3151218e-04 9.6708393e-01 2.3404445e-04 3.2450531e-02]
 [9.7959220e-02 6.6721952e-01 7.1256296e-03 2.2769560e-01]]


In [41]:
flow_pred = np.argmax(predictions, axis=-1)

In [46]:
print(x_test[0].shape)
one_flow = x_test[0].reshape(-1,20,128,1)
print(one_flow.shape)
print(one_flow)

predictions_1flow = model.predict(one_flow)
one_flow_pred = int(np.argmax(predictions_1flow, axis=-1))
print("Label", one_flow_pred)

(20, 128, 1)
(1, 20, 128, 1)
[[[[0.0627451 ]
   [0.03921569]
   [0.58431373]
   ...
   [0.07843137]
   [0.49803922]
   [0.23921569]]

  [[0.0627451 ]
   [0.03921569]
   [0.64705882]
   ...
   [0.43529412]
   [0.48235294]
   [0.1372549 ]]

  [[0.0627451 ]
   [0.03921569]
   [0.62745098]
   ...
   [0.18431373]
   [0.83137255]
   [0.31764706]]

  ...

  [[0.0627451 ]
   [0.00392157]
   [0.02352941]
   ...
   [0.56078431]
   [0.41568627]
   [1.        ]]

  [[0.0627451 ]
   [0.        ]
   [0.99215686]
   ...
   [0.40392157]
   [0.56078431]
   [0.43137255]]

  [[0.0627451 ]
   [0.00392157]
   [0.00392157]
   ...
   [0.98039216]
   [0.30980392]
   [0.98823529]]]]
1/1 [==============================] - 0s 19ms/step
Label 1


In [43]:
label_dict ={'FileTransfer': 0, 'Music': 1, 'VoIP': 2, 'Youtube': 3}

In [44]:
for key, value in label_dict.items():
    if one_flow_pred == value:
        print(key)


Music


In [45]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report


print('\nAccuracy: {:.2f}\n'.format(accuracy_score(flow_label, flow_pred)))

ValueError: Found input variables with inconsistent numbers of samples: [2780, 695]